# Demo: DQG Mappings
This notebook demonstrates the correctness of the Q and G mappings used for the spatial v2RDM.

## Contents:
1. [Constructing ¹D and ¹Q](#section1)
2. [Constructing ²D](#section2)
3. [Demonstrating the correctness of mappings $f_Q$ and $f_G$](#section3)
4. [CVXPY functions are equivalent to the Numpy functions](#section4)

In [17]:
# Setup
import numpy as np
from src.molecule_helper import *
from src.dqg import *

tolerance = 1e-8
is_positive = lambda X: np.all(np.linalg.eigvals(X) + tolerance >= 0)

## 1) Constructing ¹D and ¹Q <a class="anchor" id="section1"></a>
We construct $^1D$ and $^1Q$ using the second quantization operators provided by OpenFermion:
$$^2D^{i}_{j} = \bra{\psi} \hat{a}^\dagger_i \hat{a}_j \ket{\psi}$$
$$^2Q^{i}_{j} = \bra{\psi} \hat{a}_i \hat{a}^\dagger_j \ket{\psi}$$

The resulting 1-RDM adheres to the following properties (as required [1,2]):

(i) $^1D \succeq 0$, $^1Q \succeq 0$ (positivity)

(ii) $^1Q^i_j = \delta ^i_j - {}^1D^i_j$

In [18]:
molecule = create_molecule("HYDROGEN_CHAIN", 1.0)
hamiltonian = get_hamiltonian(molecule)
gs_energy, gs = get_groundstate(hamiltonian)
r = molecule.n_orbitals
n_qubits = molecule.n_qubits
N = molecule.n_electrons

In [19]:
print(make_summary(molecule))

Molecule:  Hydrogen_chain_H4_1.0
Electrons: 4         
Orbitals:  4         
Qubits:    8         
FCI:       -2.1663874486347607
Basis:     sto-3g    



In [20]:
# Compute D1
D1 = computeD1(molecule, gs)
Q1 = computeQ1(molecule, gs)

In [21]:
# 1) Positivity
print("D1 >> 0 ?", is_positive(D1))
print("Q1 >> 0 ?", is_positive(Q1))

D1 >> 0 ? True
Q1 >> 0 ? True


In [22]:
# 2) Mappings
Q1_map = np.eye(D1.shape[0]) - D1
print("Q1 == Q(D1) ?", np.allclose(Q1, Q1_map))

Q1 == Q(D1) ? True


## 2) Constructing ²D <a class="anchor" id="section2"></a>
We construct ${}^2D$ using the second quantization operators provided by OpenFermion:
$^2D^{ij}_{kl} = \bra{\psi} \hat{a}^\dagger_i \hat{a}^\dagger_j \hat{a}_l \hat{a}_k \ket{\psi}$

The resulting 2-RDM adheres to the following properties (as required [1, 2]).

(i) ${}^2D \succeq 0$ (positivity)

(ii) $\text{Tr}({}^2D) = N(N-1)$ where $N$ is the number of electrons (trace condition/normalization*)

(iii) ${}^2D^{ij}_{kl} = -{}^2D^{ji}_{kl}$ and ${}^2D^{ij}_{kl} = -{}^2D^{ij}_{lk}$ (anti-symmetry)

(iv) ${}^2D^{ij}_{kl} = ({}^2D^{kl}_{ij})^\dagger$ (hermiticity)

(v) $^1D^i_k = \frac{1}{N-1} \sum_j \, ^2D^{i,j}_{k,j}$


${}^*$Note: in OpenFermion's normalization convention the trace condition does not have the prefactor of 1/2.

**References:**

[1] Nakata et al. _Variational calculations of fermion second-order reduced density matrices by semidefinite programming algorithm_. J. Chem. Phys. 114, 8282 (2001); doi: 10.1063/1.136

[2] A. Eugene DePrince III. _Variational Determination of the Two-Electron Reduced Density Matrix: A Tutorial Review_. 2023. arXiv:2310.10746v3.

[3] David A. Mazziotti et al. _Reduced-density-matrix Mechanics: with Application to Many-electron Atoms and Molecules_, John Wiley & Sons, Inc. (2007). ISBN: 978-0-471-79056-3.
 

In [23]:
# Compute D2
D2 = computeD2(molecule, gs)

/Users/annika/Masters/spatial-mv2RDM/src/dqg.py:116: ComplexWarning: Casting complex values to real discards the imaginary part
  M2[i, j, k, l] = expectation(sparse, ground_state)


In [25]:
print("D2 >> 0 ?", is_positive(D2))

D2 >> 0 ? True


In [26]:
# 2) Normalisation
print("Tr(D2) =", np.trace(D2))
print("Expected:", N * (N - 1))

Tr(D2) = 11.999999999999993
Expected: 12


In [27]:
# 3) Anti-symmetry
Dijkl = D2.reshape(n_qubits, n_qubits, n_qubits, n_qubits)
Djikl = Dijkl.transpose(1, 0, 2, 3)
Dijlk = Dijkl.transpose(0, 1, 3, 2)

print("Dijkl == -Djikl ?", np.array_equal(Dijkl, -Djikl))
print("Dijkl == -Dijlk ?", np.array_equal(Dijkl, -Dijlk))

Dijkl == -Djikl ? True
Dijkl == -Dijlk ? True


In [29]:
# 4) Hermiticity
Dklij = Dijkl.conj().transpose(2, 3, 0, 1)

print("Dijkl == Dklij ?", np.array_equal(Dijkl, Dklij))

Dijkl == Dklij ? True


In [31]:
# 5) 1RDM
D1_map = D1_ik_numpy(D2, n_qubits, N)
print("D1 == D1_ik(D2) ?", np.allclose(D1_map, D1))

D1 == D1_ik(D2) ? True


## 3) The mappings $f_Q$ and $f_G$ return the expected ${}^2Q$ and ${}^2G$ <a class="anchor" id="section3"></a>
We can calculate ${}^2D$, ${}^2Q$ and ${}^2G$ using OpenFermion:

${}^2Q^{ij}_{kl} = \bra{\psi} \hat{a}_i \hat{a}_j \hat{a}^\dagger_l \hat{a}^\dagger_k \ket{\psi}$

${}^2G^{ij}_{kl} = \bra{\psi} \hat{a}^\dagger_i \hat{a}_j \hat{a}^\dagger_l \hat{a}_k \ket{\psi}$

and then apply the functions in `dqg.py` to ${}^2D$ to get back the ${}^2Q$ and ${}^2G$ constructed using OpenFermion.

Also test the relation between $^1Q$ and $^2Q$:

(i) $^1Q^i_k = \frac{1}{r-N-1} \sum_j \, ^2Q^{i,j}_{k,j}$

**References:**

[1] David A. Mazziotti et al. _Reduced-density-matrix Mechanics: with Application to Many-electron Atoms and Molecules_, John Wiley & Sons, Inc. (2007). ISBN: 978-0-471-79056-3.


In [32]:
Q2 = computeQ2(molecule, gs)
G2 = computeG2(molecule, gs)

In [33]:
# 1) Positivity (also required for Q and G)
print("Q >> 0 ?", is_positive(Q2))
print("G >> 0 ?", is_positive(G2))

Q >> 0 ? True
G >> 0 ? True


In [34]:
# 2) Mappings
Q2_map = Q2_numpy(D2, n_qubits, N)
G2_map = G2_numpy(D2, n_qubits, N)

print("Q2 == Q(D2) ?", np.allclose(Q2, Q2_map))
print("G2 == G(D2) ?", np.allclose(G2, G2_map))

Q2 == Q(D2) ? True
G2 == G(D2) ? True


In [35]:
# 3) Q1 mapping
Q1_from_Q2 = Q1_ik_numpy(Q2, molecule.n_qubits, molecule.n_electrons)
print("Q1 == Q1_ik(Q2) ?", np.allclose(Q1, Q1_from_Q2))

Q1 == Q1_ik(Q2) ? True


## 4) CVXPY functions are equivalent to the Numpy versions <a class="anchor" id="section4"></a>

In [36]:
import cvxpy as cp

D_cvx = cp.Parameter((molecule.n_qubits**2, molecule.n_qubits**2))
D_cvx.value = D2

In [37]:
D1_cvx = D1_ik_cvxpy(D_cvx, molecule.n_qubits, molecule.n_electrons).value
print("D1s match?", np.allclose(D1_cvx, D1_map))

D1s match? True


In [38]:
Q_cvx = Q2_cvxpy(D_cvx, molecule.n_qubits, molecule.n_electrons).value
print("Qs match?", np.allclose(Q_cvx, Q2_map))

Qs match? True


In [39]:
G_cvx = G2_cvxpy(D_cvx, molecule.n_qubits, molecule.n_electrons).value
print("Gs match?", np.allclose(G_cvx, G2_map))

Gs match? True


___